In [77]:
import os
from random import sample
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Must be set before importing TF to supress messages
os.environ["CUDA_VISIBLE_DEVICES"]= '2'

import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger
import numpy as np
from utils.loader import DataLoader
from utils.tools import test_model, insert_attention_layer_in_keras
from utils.data_sampler import CustomDataGenerator, CustomIterator
from utils.configs import config
from typing import List
import pymf

def load_VGG_model(img_height: int, img_width: int, lr: int, loss: tf.keras.losses.Loss, metrics: List[str], trainable: True) -> tf.keras.Model:
    """ Loads VGG-16 model.

    Args:
        img_height (int): Image height.
        img_width (int): Image width.
        lr (int): Learning rate.
        loss (tf.keras.losses.Loss): Model loss.
        metrics (List[str]): Training metrics.
        trainable (True): Set if model weights should be kept frozen or not.

    Returns:
        tf.keras.Model: TensorFlow VGG-16 model.
    """
    model = tf.keras.applications.vgg16.VGG16(input_shape=(img_height, img_width, 3))
    model.trainable = trainable
    model.compile(optimizer=tf.keras.optimizers.Adam(lr, epsilon=0.1),
                loss=loss,
                metrics=metrics)

    return model

def train_model(model: tf.keras.Model, train_set: CustomIterator, val_set: CustomIterator, epochs: int, batch_size: int, callbacks=None):
    """ Train the model. 

    Args:
        train_set (CustomIterator): Training data.
        val_set (CustomIterator): Validation data.
        epochs (int): Number of epochs to train for.
        callbacks (_type_, optional): Callbacks for loggers. Defaults to None.

    Returns:
        history: Model training history information.
    """
    history = model.fit(train_set, validation_data=val_set, validation_freq=1, epochs=epochs, steps_per_epoch=train_set.n//batch_size, validation_steps=val_set.n//batch_size if val_set is not None else None, verbose=1, callbacks=callbacks)

    return history

- Ensure sampling correct during training
- Use same training params as Freddie paper
- Early stopping once validation loss increases

In [113]:
# Set configs
img_height = 224
img_width = 224
batch_size = 128
epochs = 1000
lr = 3e-4
log_path = os.path.join(config['logs_path'], 'vgg_baseline_attention_model_256.csv')

In [ ]:
# Set augmentation and pre-processing
train_datagen = CustomDataGenerator(
                horizontal_flip=True,
                validation_split=0.1,
                preprocessing_function=tf.keras.applications.vgg16.preprocess_input, dtype=tf.float32)
test_datagen = CustomDataGenerator(
                preprocessing_function=tf.keras.applications.vgg16.preprocess_input, dtype=tf.float32)

# Load ImageNet dataset with the VGG augmentation
loader = DataLoader(batch_size, (img_height, img_width))
train_set = loader.load_train_set(aug_train=train_datagen, class_mode='categorical', shuffle=True)
val_set = loader.load_val_set(aug_val=train_datagen, class_mode='categorical', shuffle=True)
test_set = loader.load_test_set(aug_test=test_datagen, set_batch_size=False)

In [159]:
# Load pre-trained VGG-16 model
tf.keras.backend.clear_session()
model = tf.keras.models.load_model('models/vgg_trained')
model.trainable = False

In [160]:
# Get layer kernel
kernel = model.get_layer('block4_conv3').kernel
flat_kernel = tf.reshape(kernel, [-1, kernel.shape[-1]]).numpy()

In [161]:
# Get projection matrix via SNMF
n_comp = 256

nmf = pymf.SNMF(flat_kernel, num_bases=n_comp)
nmf.factorize(niter=400)
p_mat = nmf.H

In [162]:
# Insert attention layer
model = insert_attention_layer_in_keras(p_mat, model, ['block5_conv1'])

In [168]:
weight_dict

{17: <tf.Variable 'attention_block5_conv1/seeds:0' shape=(1, 17) dtype=float32, numpy=
 array([[ 1.09064765e-02,  1.18806930e-02,  4.10769353e-05,
          1.94557942e-03,  8.69130017e-04,  1.36443144e-02,
          1.22951102e-02,  8.34141206e-03,  6.67547924e-04,
          2.26908904e-02,  4.72601317e-03,  5.30491397e-03,
          1.11149158e-02,  4.40227846e-03,  4.36795782e-03,
         -0.00000000e+00,  8.11986066e-03]], dtype=float32)>,
 42: <tf.Variable 'attention_block5_conv1/seeds:0' shape=(1, 42) dtype=float32, numpy=
 array([[4.3946380e-05, 7.9736157e-05, 2.7587002e-05, 3.5155084e-05,
         3.3212738e-05, 4.1611118e-05, 6.6966997e-05, 6.9196307e-05,
         5.3129468e-05, 5.3348784e-05, 8.4001978e-05, 4.0101255e-05,
         5.3345429e-05, 4.4363493e-05, 6.0484261e-05, 3.4808745e-05,
         6.9153110e-05, 4.8986360e-05, 4.9627008e-05, 1.9349829e-05,
         3.9913215e-05, 8.6844957e-05, 4.1048217e-05, 9.7548887e-05,
         1.4989413e-05, 5.0479197e-05, 4.9211681e-

In [127]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

In [36]:
# Enable subsampling equal to standard size
train_set.set_subsampling(size=1170*2, force_class_sampling=True)

In [119]:
# Train and use CSV logger to store logs
if not os.path.exists(log_path):
    with open(log_path, "w") as my_empty_csv: pass

csv_logger = CSVLogger(log_path, separator=',', append=False)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
train_history = train_model(model=model, train_set=train_set, val_set=val_set, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger, early_stop])

Epoch 1/1000
18/18 [==============================] - 322s 19s/step - loss: 0.6138 - accuracy: 0.8236 - val_loss: 0.7642 - val_accuracy: 0.7892
Epoch 2/1000
18/18 [==============================] - 322s 19s/step - loss: 0.6785 - accuracy: 0.8141 - val_loss: 0.7639 - val_accuracy: 0.7900
Epoch 3/1000
18/18 [==============================] - 324s 19s/step - loss: 0.7259 - accuracy: 0.8039 - val_loss: 0.7605 - val_accuracy: 0.7904
Epoch 4/1000
18/18 [==============================] - 319s 19s/step - loss: 0.7055 - accuracy: 0.8029 - val_loss: 0.7634 - val_accuracy: 0.7908
Epoch 5/1000
18/18 [==============================] - 322s 19s/step - loss: 0.6921 - accuracy: 0.7926 - val_loss: 0.7591 - val_accuracy: 0.7918
Epoch 6/1000
18/18 [==============================] - 319s 19s/step - loss: 0.7015 - accuracy: 0.7900 - val_loss: 0.7649 - val_accuracy: 0.7907
Epoch 7/1000
18/18 [==============================] - 320s 19s/step - loss: 0.6689 - accuracy: 0.8141 - val_loss: 0.7626 - val_accuracy:

Took 16, 16, 14, 18 for convergence

9, 10, 10, 11

In [103]:
model.save_weights('models/baselines2/baseline90')

Test all models

In [128]:
model.load_weights(f'models/baselines/baseline256')

In [102]:
test_model(model, test_set)


Predicting on test-set...
48238/48238 [==============================] - 239s 5ms/step
Computing accuracy...

-----------------------------------------
Model Accuracy on test-set: 0.6979766988681123
-----------------------------------------

